### Code for generating an acquisition series in pycromanager that takes a 'burst' and a z-stack in 2 channels a number of times

It is important to first set up the micromanager config properly! Although the only real important part for this code is knowing what channel group you have and what your channels are

see more: https://pycro-manager.readthedocs.io/en/latest/user_guide.html 

In [26]:
from pycromanager import Acquisition, Core, JavaObject
import numpy as np

core = Core()
print(core) # this just makes sure you're connected to micromanager

### Load position list from file
Save position list from uManager and load into pycromanager (set 'fp')

In [63]:
fp = 'C:\\Users\\ar138\\Desktop\\Research\\Codes\\pycromanage\\test_pos.pos'

positionList = JavaObject('org.micromanager.PositionList')

positionList.load(fp)
nb_positions = positionList.get_number_of_positions()
position_list = []
for i in range(nb_positions):
    position = positionList.get_position(i)
    x = position.get_x()
    y = position.get_y()
    z = position.get_z()
    pos = [x, y, z]
    position_list.append(pos)
display(position_list)

[[1.5, 1.5, 2],
 [1.5, -49.695, 4],
 [-49.695, 1.5, 4],
 [-49.695, 103.89, 4],
 [-152.085, 103.89, 11]]

### Set acquisition settings
Define the parameters for the imaging session

In [64]:

num_f = 10        # number of frames per 'burst'
exp1 = 20         # exposure time (ms) of the z-stack frames
exp2 = 20         # exposure time (ms) of the burst frames
sub_interval = 10 # interval between subsets (sec)
num_subs = 10     # number of times to repeat
z_hrange = 2.5    # +/- in um from relative z
z_interval = 0.5  # z-stack step size (um)

chgroup = 'Channel' # Name of the channel group in MicroManager
ch1 = 'FITC'        # Channel for the z-stack
ch2 = 'DAPI'        # Channel for the burst

### Generate the event series
Loops over subsets, positions to generate burst & z-stack at each

Creates a list 'events' to give to the acquisition manager

In [69]:

events = []
for sub in range(num_subs):
    
    for pos_ind, [x,y,z] in enumerate(position_list):
        
        for frame in range(num_f):
            
            evt = {
                    'axes': {'subset': sub, 'time': frame, 'channel': ch2, 'z':0, 'p':pos_ind}, 
                    'config_group': [chgroup, ch2], 
                    'min_start_time' : sub * sub_interval,
                    'exposure': exp2,
                    'z' : z,
                    'x':x,
                    'y':x,
            }
            
            events.append(evt)
        
        for z_ind,z_pos in enumerate(np.arange(start=-z_hrange, stop=z_hrange, step=z_interval)):
            
            evt = {
                    'axes': {'subset': sub, 'time': num_f, 'channel': ch1, 'z':z_ind, 'p':pos_ind}, 
                    'config_group': [chgroup, ch1], 
                    'min_start_time' : sub * sub_interval,
                    'exposure': exp1,
                    'z': z+z_pos,
                    'x':xy,
                    'y':xy,
            }
            events.append(evt)

            
print('First event: ', events[0])   # take a look at your first & last event
print('Last event: ',events[-1])

First event:  {'axes': {'subset': 0, 'time': 0, 'channel': 'DAPI', 'z': 0, 'p': 0}, 'config_group': ['Channel', 'DAPI'], 'min_start_time': 0, 'exposure': 20, 'z': 2, 'x': 1.5, 'y': 1.5}
Last event:  {'axes': {'subset': 9, 'time': 10, 'channel': 'FITC', 'z': 9, 'p': 4}, 'config_group': ['Channel', 'FITC'], 'min_start_time': 90, 'exposure': 20, 'z': 13.0, 'x': 1.5, 'y': 1.5}


### Acquire

In [4]:
# Take the acquisition

with Acquisition(directory='Downloads', name='test_aq') as acq:
    
    acq.acquire(events)